In [1]:
#Libraries

#Python Libs
import sys
import os
import glob
import traceback
from datetime import datetime
import time
from geopy import distance


#Data Analysis Libs
import pandas as pd
import numpy as np

In [2]:
#Functions
def select_input_files(enh_buste_base_path,init_date,fin_date,suffix):
        selected_files = []
        all_files = glob.glob(os.path.join(enh_buste_base_path,"*"))

        for file_ in all_files:
                try:
                        file_date = pd.to_datetime(file_.split('/')[-1],format=('%Y_%m_%d' + suffix  + '.csv'))
                        if (file_date >= init_date) and (file_date <= fin_date):
                                selected_files.append((file_,file_date))
                except:
                        continue

        return sorted(selected_files)

def get_gtfs_path(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2019-05-13", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'campina-gtfs-2019'
    else:
        return 'campina-gtfs-2017'

## Main

In [3]:
otp_suggestions_filepath = 'data/output/2019_05_13_bus_trips_otp_itineraries.csv'
bus_trips_folderpath = 'data/input'
gtfs_base_folderpath = 'data/input'
output_folderpath = 'data/output'

In [4]:
file_date_str = otp_suggestions_filepath.split('/')[-1].split('_bus_trips_')[0]
file_date = pd.to_datetime(file_date_str,format='%Y_%m_%d')
print "Processing File:", otp_suggestions_filepath

Processing File: data/output/2019_05_13_bus_trips_otp_itineraries.csv


In [5]:
    # Extracting itinerary part name for later use
    itinerary_part_name = otp_suggestions_filepath.split('/')[-1].split('_')[5]
    # Read OTP Suggestions
    otp_suggestions_raw = pd.read_csv(otp_suggestions_filepath, parse_dates=['date','otp_start_time','otp_end_time'])

    if len(otp_suggestions_raw) == 0:
        print "Zero OTP suggestions found."
        print "Skipping next steps..."
        exit(0)


In [6]:
def prepare_otp_data(otp_data):
        #Fixing prefix
        otp_data.columns = otp_data.columns.str.replace('otp_','')
        otp_data = otp_data.add_prefix('otp_')
        
        #Fixing Timezone difference - when needed
        otp_data['otp_start_time'] = otp_data['otp_start_time'] - pd.Timedelta('10800 s')
        otp_data['otp_end_time'] = otp_data['otp_end_time'] - pd.Timedelta('10800 s')
        
        #Adjusting route format to have 3 numbers
        otp_data['otp_route'] = otp_data['otp_route'].astype(str)
        otp_data['otp_route'] = np.where(otp_data['otp_mode'] == 'BUS',
                            otp_data['otp_route'].astype(str).str.replace("\.0",'').str.zfill(3),
                            otp_data['otp_route'])

        return otp_data

In [7]:
    # Prepare OTP data for analysis
    otp_suggestions = prepare_otp_data(otp_suggestions_raw)    
    
    # Read stops data
    stops_filepath = gtfs_base_folderpath + os.sep + get_gtfs_path(file_date) + os.sep + 'stops.txt'
    stops_df = pd.read_csv(stops_filepath)
    
    # Adding Parent Stop data to OTP Suggestions TODO
    stops_parent_stations = stops_df[['stop_id','parent_station']]
    otp_suggestions = otp_suggestions.merge(stops_parent_stations.add_prefix('from_'),
                                                left_on='otp_from_stop_id',
                                                right_on='from_stop_id',
                                                how='left') \
                                        .merge(stops_parent_stations.add_prefix('to_'),
                                                left_on='otp_to_stop_id',
                                                right_on='to_stop_id',
                                                how='left') \
                                        .drop(['from_stop_id','to_stop_id'], axis=1) \
                                        .rename(index=str, columns={'from_parent_station':'otp_from_parent_station',
                                                                    'to_parent_station':'otp_to_parent_station'})
        
    otp_suggestions_bus_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'BUS']
    otp_suggestions_walk_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'WALK']

In [8]:
otp_suggestions.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,2019-05-13,491551.0,1,1,2019-05-13 05:13:45,2019-05-13 05:13:46,WALK,nan,NaN,NaN,0.016667,NaN,NaN
1,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN
2,2019-05-13,491551.0,1,3,2019-05-13 05:43:31,2019-05-13 05:47:06,WALK,nan,NaN,NaN,3.583333,NaN,NaN
3,2019-05-13,491551.0,1,4,2019-05-13 05:47:40,2019-05-13 05:56:33,BUS,903B,386641.0,386291.0,8.883333,NaN,NaN
4,2019-05-13,491551.0,1,5,2019-05-13 05:56:34,2019-05-13 05:56:35,WALK,nan,NaN,NaN,0.016667,NaN,NaN


## Read and Prepare Bus Trip Data

In [9]:
bus_trips_filepath = bus_trips_folderpath + os.sep + file_date_str + '_bus_trips.csv'
bus_trips = pd.read_csv(bus_trips_filepath, dtype={'route': object},parse_dates=['gps_datetime']) \
                                        .sort_values(['route','busCode','tripNum','gps_datetime']) \
                                        .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3))  \
                                        .drop_duplicates()

In [10]:
bus_trips.head()

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,gps_datetime,stopPointId,problem
5900,004A,1,90026,low_frequency,173,-7.23043,-35.87588,9535.0,3005,11553,-7.230581,-35.87565,30.42416,2019-05-13 06:45:24,385835,NO_PROBLEM
5962,004A,1,90026,low_frequency,45,-7.22907,-35.87689,3163.0,3005,11678,-7.228993,-35.87696,11.529692,2019-05-13 06:45:55,386394,NO_PROBLEM
6129,004A,1,90026,low_frequency,74,-7.22220,-35.87565,4030.0,3005,-,-,-,-,2019-05-13 06:47:47,386129,BETWEEN
6244,004A,1,90026,low_frequency,86,-7.21908,-35.87549,4429.0,3005,12300,-7.219028,-35.8755,5.8864255,2019-05-13 06:48:59,386131,NO_PROBLEM
6486,004A,1,90026,low_frequency,152,-7.22434,-35.87674,8788.0,3005,-,-,-,-,2019-05-13 06:51:35,385833,BETWEEN


In [48]:
bus_trips[(bus_trips.route == "944") & (bus_trips.stopPointId == 491551.0)].head()

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,gps_datetime,stopPointId,problem
4266,944,2,72976,high_frequency,40,-7.28659,-35.89567,1398.0,1002,-,-,-,-,2019-05-13 06:29:21,491551,BETWEEN
12536,944,3,72976,high_frequency,40,-7.28659,-35.89567,1398.0,1002,24653,-7.286584,-35.89571,4.4620376,2019-05-13 07:44:09,491551,NO_PROBLEM
21922,944,4,72976,high_frequency,40,-7.28659,-35.89567,1398.0,1002,-,-,-,-,2019-05-13 08:54:42,491551,BETWEEN
28785,944,5,72976,high_frequency,40,-7.28659,-35.89567,1398.0,1002,-,-,-,-,2019-05-13 10:08:54,491551,BETWEEN
38165,944,6,72976,high_frequency,40,-7.28659,-35.89567,1398.0,1002,-,-,-,-,2019-05-13 11:25:35,491551,BETWEEN


In [11]:
bus_trips_clean = bus_trips.filter(['route','busCode','tripNum','stopPointId','gps_datetime'])

In [12]:
bus_trips_clean.head()

,route,busCode,tripNum,stopPointId,gps_datetime
5900,004A,3005,1,385835,2019-05-13 06:45:24
5962,004A,3005,1,386394,2019-05-13 06:45:55
6129,004A,3005,1,386129,2019-05-13 06:47:47
6244,004A,3005,1,386131,2019-05-13 06:48:59
6486,004A,3005,1,385833,2019-05-13 06:51:35


In [25]:
bus_trips_clean[bus_trips_clean.route == "944"]

,route,busCode,tripNum,stopPointId,gps_datetime
278,944,1002,1,386660,2019-05-13 05:22:52
287,944,1002,1,386659,2019-05-13 05:23:22
298,944,1002,1,386657,2019-05-13 05:23:53
338,944,1002,1,386656,2019-05-13 05:25:55
378,944,1002,1,497506,2019-05-13 05:27:16
401,944,1002,1,497505,2019-05-13 05:27:57
443,944,1002,1,497504,2019-05-13 05:29:29
465,944,1002,1,491559,2019-05-13 05:30:30
507,944,1002,1,385793,2019-05-13 05:32:02
520,944,1002,1,385790,2019-05-13 05:32:32


In [13]:
bus_trips_clean.dtypes

route                   object
busCode                  int64
tripNum                  int64
stopPointId              int64
gps_datetime    datetime64[ns]
dtype: object

In [14]:
otp_suggestions_bus_legs.dtypes

otp_date                   datetime64[ns]
otp_user_trip_id                  float64
otp_itinerary_id                    int64
otp_leg_id                          int64
otp_start_time             datetime64[ns]
otp_end_time               datetime64[ns]
otp_mode                           object
otp_route                          object
otp_from_stop_id                  float64
otp_to_stop_id                    float64
otp_duration_mins                 float64
otp_from_parent_station           float64
otp_to_parent_station             float64
dtype: object

## Identify Possible Matches between OTP Itineraries and Bus Trips Observed Data

In [59]:
scheduled_itin_observed_o = otp_suggestions_bus_legs.merge(bus_trips_clean.add_prefix('bt_'),
                                left_on=['otp_route','otp_from_stop_id'],
                                right_on=['bt_route','bt_stopPointId'],
                                how='inner') \
                                .drop(['bt_route','bt_stopPointId'], axis=1) \
                                .rename(index=str, columns={'bt_gps_datetime':'bt_start_time',
                                                            'bt_tripNum':'bt_trip_num',
                                                            'bt_busCode':'bt_bus_code'}) \
                                .assign(sched_obs_start_timediff = 
                                        lambda x: np.absolute(x['bt_start_time'] - x['otp_start_time']))

In [50]:
bus_trips_clean[(bus_trips_clean.route == "944") & (bus_trips_clean.stopPointId == 491551.0)].sort_values(by=['gps_datetime'])

,route,busCode,tripNum,stopPointId,gps_datetime
4266,944,1002,2,491551,2019-05-13 06:29:21
6146,944,1052,2,491551,2019-05-13 06:47:57
8084,944,1096,2,491551,2019-05-13 07:06:16
12536,944,1002,3,491551,2019-05-13 07:44:09
14182,944,1051,2,491551,2019-05-13 07:56:49
15108,944,1052,3,491551,2019-05-13 08:04:18
16849,944,1096,3,491551,2019-05-13 08:18:15
21922,944,1002,4,491551,2019-05-13 08:54:42
24607,944,1096,5,491551,2019-05-13 09:34:54
28785,944,1002,5,491551,2019-05-13 10:08:54


In [51]:
scheduled_itin_observed_o.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station,bt_bus_code,bt_trip_num,bt_stopPointId,bt_start_time,sched_obs_start_timediff
0,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN,1002,2,491551,2019-05-13 06:29:21,01:15:34
1,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN,1002,3,491551,2019-05-13 07:44:09,02:30:22
2,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN,1002,4,491551,2019-05-13 08:54:42,03:40:55
3,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN,1002,5,491551,2019-05-13 10:08:54,04:55:07
4,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN,1002,6,491551,2019-05-13 11:25:35,06:11:48


In [56]:
scheduled_itin_observed_o[(scheduled_itin_observed_o.otp_route == "944") & (scheduled_itin_observed_o.otp_start_time == '2019-05-13 14:05:47')][['otp_itinerary_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           "otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff'])

,otp_itinerary_id,otp_start_time,otp_route,otp_from_stop_id,otp_from_stop_id,bt_start_time,sched_obs_start_timediff
9407,229,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
9360,228,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
9454,230,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
9266,226,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
9406,229,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:47:41,00:18:06
9453,230,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:47:41,00:18:06
9359,228,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:47:41,00:18:06
9265,226,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:47:41,00:18:06
9287,226,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
9381,228,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46


In [61]:
scheduled_itin_observed_od = scheduled_itin_observed_o.merge(bus_trips_clean.add_prefix('bt_'),
                                left_on=['otp_route','bt_bus_code','bt_trip_num','otp_to_stop_id'],
                                right_on=['bt_route','bt_busCode','bt_tripNum','bt_stopPointId'],
                                how='inner') \
                                .drop(['bt_route','bt_stopPointId'], axis=1) \
                                .rename(index=str, columns={'bt_gps_datetime':'bt_end_time'}) \
                                .assign(sched_obs_end_timediff = 
                                        lambda x: np.absolute(x['bt_end_time'] - x['otp_end_time'])) \
                                .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','sched_obs_start_timediff','sched_obs_end_timediff'])

In [17]:
len(scheduled_itin_observed_od)

26845

In [18]:
scheduled_itin_observed_od['bt_duration_mins'] = (scheduled_itin_observed_od['bt_end_time'] - scheduled_itin_observed_od['bt_start_time'])/pd.Timedelta(minutes=1)
scheduled_itin_observed_od = scheduled_itin_observed_od[scheduled_itin_observed_od['bt_duration_mins'] > 0]

In [19]:
len(scheduled_itin_observed_od)

22121

In [20]:
scheduled_itin_observed_od_full = pd.concat([scheduled_itin_observed_od,otp_suggestions_walk_legs], sort=False)

In [23]:
bus_trips_clean[(bus_trips['route'] == '944') & (bus_trips['stopPointId'] == 491551)].sort_values(['gps_datetime'])

,route,busCode,tripNum,stopPointId,gps_datetime
4266,944,1002,2,491551,2019-05-13 06:29:21
6146,944,1052,2,491551,2019-05-13 06:47:57
8084,944,1096,2,491551,2019-05-13 07:06:16
12536,944,1002,3,491551,2019-05-13 07:44:09
14182,944,1051,2,491551,2019-05-13 07:56:49
15108,944,1052,3,491551,2019-05-13 08:04:18
16849,944,1096,3,491551,2019-05-13 08:18:15
21922,944,1002,4,491551,2019-05-13 08:54:42
24607,944,1096,5,491551,2019-05-13 09:34:54
28785,944,1002,5,491551,2019-05-13 10:08:54


In [57]:
scheduled_itin_observed_od_full.head() \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,...,otp_to_parent_station,bt_bus_code,bt_trip_num,bt_start_time,sched_obs_start_timediff,bt_busCode,bt_tripNum,bt_end_time,sched_obs_end_timediff,bt_duration_mins
0,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,...,NaN,1002.0,2.0,2019-05-13 06:29:21,01:15:34,1002.0,2.0,2019-05-13 07:11:02,01:27:31,41.683333
2520,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,...,NaN,1052.0,2.0,2019-05-13 06:47:57,01:34:10,1052.0,2.0,2019-05-13 07:31:28,01:47:57,43.516667
3480,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,...,NaN,1096.0,2.0,2019-05-13 07:06:16,01:52:29,1096.0,2.0,2019-05-13 07:47:36,02:04:05,41.333333
120,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,...,NaN,1002.0,3.0,2019-05-13 07:44:09,02:30:22,1002.0,3.0,2019-05-13 08:23:26,02:39:55,39.283333
1200,2019-05-13,491551.0,1,2,2019-05-13 05:13:47,2019-05-13 05:43:31,BUS,944,491551.0,386521.0,...,NaN,1051.0,2.0,2019-05-13 07:56:49,02:43:02,1051.0,2.0,2019-05-13 09:49:33,04:06:02,112.733333


In [63]:
scheduled_itin_observed_od_full_clean[(scheduled_itin_observed_od_full_clean.otp_route == "944") & (scheduled_itin_observed_od_full_clean.otp_start_time == '2019-05-13 14:05:47')][['otp_itinerary_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           "otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff'])

,otp_itinerary_id,otp_start_time,otp_route,otp_from_stop_id,otp_from_stop_id,bt_start_time,sched_obs_start_timediff
783,226,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
5847,228,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
5848,229,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
5849,230,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
8550,228,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
8551,229,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
8552,230,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
2823,226,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
8711,230,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 14:52:34,00:46:47
8710,229,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 14:52:34,00:46:47


In [62]:
scheduled_itin_observed_od_full_clean = scheduled_itin_observed_od_full \
                            .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_mode','otp_route',
                                     'bt_bus_code','bt_trip_num','otp_from_stop_id','otp_start_time',
                                     'bt_start_time','sched_obs_start_timediff','otp_to_stop_id',
                                     'otp_end_time','bt_end_time','sched_obs_end_timediff','otp_duration_mins','minimun_obs_start_time']) \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])


In [ ]:
scheduled_itin_observed_od_full_clean

In [64]:
scheduled_itin_observed_od_full_clean \
                        .groupby(['otp_itinerary_id', 'otp_leg_id']) \
                        .apply(lambda x: x.sort_values(["sched_obs_start_timediff"]))


otp_user_trip_id  otp_itinerary_id  \
otp_itinerary_id otp_leg_id                                             
1                1          0              491551.0                 1   
                 2          0              491551.0                 1   
                            2520           491551.0                 1   
                            3480           491551.0                 1   
                            120            491551.0                 1   
                            1200           491551.0                 1   
                            3600           491551.0                 1   
                            240            491551.0                 1   
                            3720           491551.0                 1   
                            360            491551.0                 1   
                            1320           491551.0                 1   
                            3840           491551.0                 1   
                            480            491551.0                 1   
                            1560           491551.0                 1   
                            3960           491551.0                 1   
                            2640           491551.0                 1   
                            1680           491551.0                 1   
                            4080           491551.0                 1   
                            2760           491551.0                 1   
                            720            491551.0                 1   
                            2880           491551.0                 1   
                            4320           491551.0                 1   
                            3000           491551.0                 1   
                            840            491551.0                 1   
                            1920           491551.0                 1   
                            4440           491551.0                 1   
                            3120           491551.0                 1   
                            2040           491551.0                 1   
                            1080           491551.0                 1   
                            4560           491551.0                 1   
...                                             ...               ...   
410              3          15874          491551.0               410   
                            16189          491551.0               410   
                            15594          491551.0               410   
                            14684          491551.0               410   
                            16154          491551.0               410   
                            15139          491551.0               410   
                            15804          491551.0               410   
                            14649          491551.0               410   
                            16119          491551.0               410   
                            15104          491551.0               410   
                            14614          491551.0               410   
                            16084          491551.0               410   
                            15069          491551.0               410   
                            15489          491551.0               410   
                            14894          491551.0               410   
                            15769          491551.0               410   
                            16049          491551.0               410   
                            15034          491551.0               410   
                            15454          491551.0               410   
                            14859          491551.0               410   
                            15419          491551.0               410   
                            14824          491551.0               410   
                    

In [69]:
scheduled_itin_observed_od_full_clean[(scheduled_itin_observed_od_full_clean.otp_route == "944") & (scheduled_itin_observed_od_full_clean.otp_start_time == '2019-05-13 14:05:47') & (scheduled_itin_observed_od_full_clean.otp_itinerary_id == 230)][['otp_itinerary_id','otp_leg_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           "otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff']).head()

,otp_itinerary_id,otp_leg_id,otp_start_time,otp_route,otp_from_stop_id,otp_from_stop_id,bt_start_time,sched_obs_start_timediff
5849,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03
8552,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:40:01,00:25:46
8711,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 14:52:34,00:46:47
10301,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:00:06,01:05:41
7121,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 12:50:51,01:14:56


In [73]:
a = scheduled_itin_observed_od_full_clean.drop_duplicates(subset=['otp_itinerary_id','otp_leg_id'])

In [75]:
a[(a.otp_route == "944") & (a.otp_start_time == '2019-05-13 14:05:47') & (a.otp_itinerary_id == 230)][['otp_itinerary_id','otp_leg_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           "otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff']).head()

,otp_itinerary_id,otp_leg_id,otp_start_time,otp_route,otp_from_stop_id,otp_from_stop_id,bt_start_time,sched_obs_start_timediff
5849,230,2,2019-05-13 14:05:47,944,491551.0,491551.0,2019-05-13 13:51:44,00:14:03


In [ ]:
# Filtering out itineraries which lost bus legs along the processing
original_suggested_itins_num_legs = otp_suggestions.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x)}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})

curr_matched_itins_num_legs = scheduled_itin_observed_od_full_clean.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(np.unique(x))}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})

complete_matched_itins = original_suggested_itins_num_legs.merge(curr_matched_itins_num_legs, how='inner')

all_complete_vehicle_legs_options = scheduled_itin_observed_od_full_clean.merge(complete_matched_itins.drop('num_legs', axis=1), how='inner')

In [ ]:
def is_new_itinerary(prev_trip_id,curr_trip_id,prev_itin_id,curr_itin_id):
    return ((prev_trip_id != curr_trip_id) | (prev_itin_id != curr_itin_id))

def choose_leg_matches(leg_matches_groups):
        colnames = leg_matches_groups.obj.columns.values
        chosen_leg_matches = pd.DataFrame(columns = colnames)
        prev_trip_id = -1
        prev_itin_id = -1
        prev_leg_mode = ""
        prev_leg_end_time = pd.NaT
        num_groups_not_survived = 0
        new_itinerary = False

        for name, group in leg_matches_groups:
            
                #print
                #print "Name:", name
                #print "Group:"
                #print group
                #print
                
                curr_trip_id = group['otp_user_trip_id'].iloc[0]
                curr_itin_id = group['otp_itinerary_id'].iloc[0]
                curr_leg_id = group['otp_leg_id'].iloc[0]
                curr_leg_mode = group['otp_mode'].iloc[0]
                
                new_itinerary = is_new_itinerary(prev_trip_id,curr_trip_id,prev_itin_id,curr_itin_id)
                if new_itinerary:
                    prev_leg_end_time = group['otp_start_time'].dt.floor('d').iloc[0]

                #if (prev_group_id == ()):
                #        prev_leg_end_time = group['bt_start_time'].dt.floor('d')[0]

                #print
                #print "Previous itinerary id:", prev_itin_id
                #print "Previous leg mode:", prev_leg_mode
                #print "Previous leg end time:", prev_leg_end_time
                #print "Current leg id:", curr_leg_id
                #print "Current leg mode:", curr_leg_mode
                #print
                #print "Original Group"
                #print group.filter(['otp_start_time','bt_start_time','bt_end_time'])
                
                if (curr_leg_mode == 'WALK'):
                    #print "Walking duration:", filtered_group['otp_duration_mins']
                    filtered_group = group.reset_index()
                    if new_itinerary: #first leg is a WALK leg
                        filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time
                    else:
                        filtered_group.loc[0,'bt_start_time'] = prev_leg_end_time
                        filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time + \
                            pd.Timedelta(minutes=np.rint(filtered_group['otp_duration_mins'].iloc[0]))
                    #print "Filtered Group"
                    #print filtered_group
                else:
                    filtered_group = group[group['bt_start_time'] > prev_leg_end_time]
                
                #print
                #print "Filtered Group"
                #print filtered_group.filter(['otp_start_time','bt_start_time','bt_end_time'])

                if (len(filtered_group) == 0):
                        #print "Group did not survive! =("
                        #print
                        #print "Previous itinerary id:", prev_itin_id
                        #print "Previous leg mode:", prev_leg_mode
                        #print "Previous leg end time:", prev_leg_end_time
                        #print "Current leg id:", curr_leg_id
                        #print "Current leg mode:", curr_leg_mode
                        #print
                        #print "Original Group"
                        #print group#.filter(['otp_start_time','bt_start_time','bt_end_time'])
                        num_groups_not_survived += 1
                        continue

                chosen_leg_match = filtered_group.sort_values('bt_start_time').iloc[0]
                
                if ((curr_leg_id == 2) & 
                    ((curr_leg_mode == 'BUS') & (prev_leg_mode == 'WALK'))):
                        #Update previous walk start/end_times
                        #print
                        #print "Chosen Leg Matches"
                        #print chosen_leg_matches.iloc[-1]
                        #print
                        chosen_leg_matches.iloc[-1,chosen_leg_matches.columns.get_loc('bt_start_time')] = chosen_leg_match['bt_start_time'] - \
                            pd.Timedelta(minutes=np.rint(chosen_leg_matches.iloc[-1].otp_duration_mins))
                        chosen_leg_matches.iloc[-1,chosen_leg_matches.columns.get_loc('bt_end_time')] = chosen_leg_match['bt_start_time']
                #print "Chosen Leg"
                #print chosen_leg_match

                chosen_leg_matches = chosen_leg_matches.append(chosen_leg_match)

                #Update variables
                #prev_group_id = name
                prev_trip_id = curr_trip_id
                prev_itin_id = curr_itin_id
                prev_leg_mode = curr_leg_mode
                prev_leg_end_time = chosen_leg_match['bt_end_time']

        #print "Number of groups which did not survive:", num_groups_not_survived
        return chosen_leg_matches.filter(colnames)

In [ ]:
# Choose best actual leg matches (based on feasibility and start time)
feasible_legs = choose_leg_matches(scheduled_itin_observed_od_full_clean.groupby(['otp_user_trip_id','otp_itinerary_id','otp_leg_id']))

if len(feasible_legs) == 0:
    print "No matches left after matching and selecting feasible bus legs."
    print "Skipping next steps..."
    exit(0)

# Filtering out itineraries which lost bus legs after feasible legs choice processing
feasible_itins_num_legs = feasible_legs.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x)}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})

feasible_complete_itins = feasible_itins_num_legs.merge(original_suggested_itins_num_legs,how='inner')

feasible_complete_itins_legs = feasible_legs.merge(feasible_complete_itins.drop('num_legs', axis=1),how='inner')

In [ ]:
def add_stops_data_to_legs(itineraries_legs,stops_locs):
    itineraries_legs_stops = itineraries_legs.merge(stops_locs, left_on='otp_from_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'from_stop_lat','stop_lon':'from_stop_lon'}) \
                                                                                .merge(stops_locations, left_on='otp_to_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'to_stop_lat','stop_lon':'to_stop_lon'}) 
    return itineraries_legs_stops

In [ ]:
stops_locations = stops_df[['stop_id','stop_lat','stop_lon']]
itineraries_legs = add_stops_data_to_legs(feasible_complete_itins_legs,stops_locations)

In [ ]:
feasible_complete_itins_legs